## 0. General Imports

In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import *
from pyspark.sql.functions import *

conf = pyspark.SparkConf().setMaster("local[*]").setAll([
                                   ('spark.executor.memory', '12g'),  # find
                                   ('spark.driver.memory','4g'), # your
                                   ('spark.driver.maxResultSize', '2G') # setup
                                  ])
# create the session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# create the context
sc = spark.sparkContext

# FIX for Spark 2.x
locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from scipy import stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## 1. Data preparation

### 1.1 Swiss production
#### 1.1.1 Data Description
The FAO Database offers a variety of datasets containing information about the production of every country for all years between 1961 and 2017.
##### Crops
<details>
  <summary>Description</summary> 
  Crop statistics are recorded for 173 products, covering the following categories: Crops Primary, Fibre Crops Primary, Cereals, Coarse Grain, Citrus Fruit, Fruit, Jute Jute-like Fibres, Oilcakes Equivalent, Oil crops Primary, Pulses, Roots and Tubers, Treenuts and Vegetables and Melons. Data are expressed in terms of area harvested, production quantity and yield. The objective is to comprehensively cover production of all primary crops for all countries and regions in the world.
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/Production_Crops_E_All_Data_(Normalized).zip
</details>

##### Crops Processed
<details>
  <summary>Description</summary>  
  The dataset covers the following commodities: Beer of barley; Cotton lint; Cottonseed; Margarine, short; Molasses; Oil, coconut (copra); Oil, cottonseed; Oil, groundnut; Oil, linseed; Oil, maize; Oil, olive, virgin; Oil, palm; Oil, palm kernel; Oil, rapeseed; Oil, safflower; Oil, sesame; Oil, soybean; Oil, sunflower; Palm kernels; Sugar Raw Centrifugal; Wine.
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Production_CropsProcessed_E_All_Data_(Normalized).zip
</details>

##### Live Animals
<details>
  <summary>Description</summary>  
  The dataset contains the following commodities and commodity aggregates thereof : Animals live n.e.s.; Asses; Beehives; Buffaloes; Camelids, other; Camels; Cattle; Chickens; Ducks; Geese and guinea fowls; Goats; Horses; Mules; Pigeons, other birds; Pigs; Rabbits and hares; Rodents, other; Sheep; Turkeys.
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Production_Livestock_E_All_Data_(Normalized).zip
</details>

##### Livestock Primary
<details>
  <summary>Description</summary>  
  The dataset contains the following commodities and commodity aggregates thereof : Beeswax; Eggs (various types); Hair, horse; Hides buffalo, fresh; Hides, cattle, fresh; Honey, natural; Meat indigenous (ass, bird nes, buffalo, camel, cattle, chicken, duck, geese, goat, horse, mule, other camelids, pig, rabbit, rodents, sheep, turkey); Meat (ass, bird nes, buffalo, camel, cattle, chicken, duck, game, goat, goose and guinea fowl, horse, mule, Meat nes, meat other camelids, Meat other rodents, pig, rabbit, sheep, turkey); Milk (buffalo, camel, cow, goat, sheep); Offals, nes; Silk-worm cocoons, reelable; Skins, furs; Skins (goat, sheep); Snails, not sea; Wool, greasy. 
    The data presented in the table are incomplete, particularly with regard to African and Asian countries.
    
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Production_LivestockPrimary_E_All_Data_(Normalized).zip
</details>

##### Livestock Processed
<details>
  <summary>Description</summary>  
  The dataset covers the following commodities: Butter and ghee, sheep milk; Butter of goat milk; Butter, buffalo milk; Butter, cow milk; Cheese of goat milk; Cheese, buffalo milk; Cheese, sheep milk; Cheese, skimmed cow milk; Cheese, whole cow milk; Cream fresh; Ghee, butteroil of cow milk; Ghee, of buffalo milk; Lard; Milk, dry buttermilk; Milk, skimmed condensed; Milk, skimmed cow; Milk, skimmed dried; Milk, skimmed evaporated; Milk, whole condensed; Milk, whole dried; Milk, whole evaporated; Silk raw; Tallow; Whey, condensed; Whey, dry; Yoghurt
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Production_LivestockProcessed_E_All_Data_(Normalized).zip
</details>

##### Production Indices
<details>
  <summary>Description</summary>  
  The dataset includes data on gross and net production indices for various food and agriculture aggregates expressed in both totals and per capita.
    The FAO indices of agricultural production show the relative level of the aggregate volume of agricultural production for each year in comparison with the base period 2004-2006. They are based on the sum of price-weighted quantities of different agricultural commodities produced after deductions of quantities used as seed and feed weighted in a similar manner.
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Production_Indices_E_All_Data_(Normalized).zip
</details>

##### Value of Agricultural Production
<details>
  <summary>Description</summary>  
  The data set includes data on gross and net production values, in constant international US\$, and gross production values, in constant and current US\$ and Local Currency Units, for various food and agriculture commodities and aggregates thereof, expressed in both total value and value per capita
</details>

<details>
  <summary>Path</summary>  
  Data/FAOSTAT/Value_of_Production_E_All_Data_(Normalized).zip
</details>

We are mainly interested in the production of all different type of food in Switzerland. For this reason we will not use "Live Animals", but rather combine the Crops and Livestock produced in Switzerland.

We will not use "Production Indices" and "Value of Agricultural Production" for now because they only contain monetary value and do not contain all data; in particular they only take into consideration Primary Crops and Livestock. We can use them as is if we want a simpler dataset.

#### 1.1.2 Data Cleaning

In [3]:
"""Load the Data"""
crops_df = pd.read_csv('Data/FAOSTAT/Production_Crops_E_All_Data_(Normalized).zip', sep=',', encoding = "ISO-8859-1")
crops_processed_df = pd.read_csv('Data/FAOSTAT/Production_CropsProcessed_E_All_Data_(Normalized).zip', sep=',', encoding = "ISO-8859-1")
livestock_primary_df = pd.read_csv('Data/FAOSTAT/Production_LivestockPrimary_E_All_Data_(Normalized).zip', sep=',', encoding = "ISO-8859-1")
livestock_processed_df = pd.read_csv('Data/FAOSTAT/Production_LivestockProcessed_E_All_Data_(Normalized).zip', sep=',', encoding = "ISO-8859-1")

In [4]:
"""Extract Switzerland Production"""
CH_code = 211 # Switzerland Area Code is 211
crops_df_CH = crops_df[crops_df['Area Code'] == CH_code]
crops_processed_df_CH = crops_processed_df[crops_processed_df['Area Code'] == CH_code]
livestock_primary_df_CH = livestock_primary_df[livestock_primary_df['Area Code'] == CH_code]
livestock_processed_df_CH = livestock_processed_df[livestock_processed_df['Area Code'] == CH_code]

In [5]:
"""Combines the dataframes into one"""
production_df_CH = pd.concat([crops_df_CH, crops_processed_df_CH, livestock_primary_df_CH, livestock_processed_df_CH])
production_df_CH.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
1501762,211,Switzerland,515,Apples,5312,Area harvested,1961,1961,ha,2700.0,F
1501763,211,Switzerland,515,Apples,5312,Area harvested,1962,1962,ha,4800.0,F
1501764,211,Switzerland,515,Apples,5312,Area harvested,1963,1963,ha,3600.0,F
1501765,211,Switzerland,515,Apples,5312,Area harvested,1964,1964,ha,4000.0,F
1501766,211,Switzerland,515,Apples,5312,Area harvested,1965,1965,ha,2478.0,NaN


In many cases we are only interested in the production quantity rather than the amount of resources needed for the production.

In [6]:
"""Only keep production quantity"""
prod_quant_df_CH = production_df_CH[production_df_CH['Element'] == 'Production']
prod_quant_df_CH.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
1501876,211,Switzerland,515,Apples,5510,Production,1961,1961,tonnes,269800.0,NaN
1501877,211,Switzerland,515,Apples,5510,Production,1962,1962,tonnes,480400.0,NaN
1501878,211,Switzerland,515,Apples,5510,Production,1963,1963,tonnes,359600.0,NaN
1501879,211,Switzerland,515,Apples,5510,Production,1964,1964,tonnes,389600.0,NaN
1501880,211,Switzerland,515,Apples,5510,Production,1965,1965,tonnes,255400.0,NaN


In [7]:
"""Check Units"""
prod_quant_df_CH['Unit'].value_counts()

tonnes       7414
Head          265
1000 Head     212
1000 No        57
Name: Unit, dtype: int64

We notice a few different units, the most frequent one being `tonnes`. We need to check what the other units correspond to to see if we should remove them.

In [8]:
"""Items expressed in Head or 1000 Head"""
prod_quant_df_CH[(prod_quant_df_CH['Unit'] == 'Head') | (prod_quant_df_CH['Unit'] == '1000 Head')]['Item'].value_counts()

Meat indigenous, goat       53
Meat indigenous, poultry    53
Meat indigenous, sheep      53
Meat indigenous, horse      53
Meat indigenous, cattle     53
Meat indigenous, rabbit     53
Meat indigenous, pig        53
Meat indigenous, turkey     53
Meat indigenous, chicken    53
Name: Item, dtype: int64

In [9]:
"""Same items expressed in tonnes"""
prod_quant_df_CH[prod_quant_df_CH['Item'].str.contains('Meat indigenous') & (prod_quant_df_CH['Unit'] == 'tonnes')]['Item'].value_counts()

Meat indigenous, goat       53
Meat indigenous, poultry    53
Meat indigenous, sheep      53
Meat indigenous, horse      53
Meat indigenous, cattle     53
Meat indigenous, rabbit     53
Meat indigenous, pig        53
Meat indigenous, turkey     53
Meat indigenous, total      53
Meat indigenous, chicken    53
Name: Item, dtype: int64

In [10]:
"""Items expressed in 1000 No"""
prod_quant_df_CH[prod_quant_df_CH['Unit'] == '1000 No']['Item'].value_counts()

Eggs, hen, in shell (number)    57
Name: Item, dtype: int64

In [11]:
"""Same items expressed in tonnes"""
prod_quant_df_CH[prod_quant_df_CH['Item'].str.contains('Eggs, hen, in shell') & (prod_quant_df_CH['Unit'] == 'tonnes')]['Item'].value_counts()

Eggs, hen, in shell    57
Name: Item, dtype: int64

We notices that the items expressed with those other units also have a values in tonnes. Thus, we can safely remove the rows with the other units as they do not provide extra information.

The following dataframe then provides the complete swiss food production in terms of tonnes from 1961 to 2017. A few items have missing data as the starting year of their report is not always similar. 

In [12]:
"""Complete Swiss food production fataframe"""
prod_quant_df_CH = prod_quant_df_CH[prod_quant_df_CH['Unit'] == 'tonnes']
prod_quant_df_CH.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
1501876,211,Switzerland,515,Apples,5510,Production,1961,1961,tonnes,269800.0,NaN
1501877,211,Switzerland,515,Apples,5510,Production,1962,1962,tonnes,480400.0,NaN
1501878,211,Switzerland,515,Apples,5510,Production,1963,1963,tonnes,359600.0,NaN
1501879,211,Switzerland,515,Apples,5510,Production,1964,1964,tonnes,389600.0,NaN
1501880,211,Switzerland,515,Apples,5510,Production,1965,1965,tonnes,255400.0,NaN


#### 1.1.3 Pending Questions
* Missing Data: Do we keep it as is? Set to 0?
* Flags: Are they useful? How would we use them?
* Items: Duplicates?

---

### 1.2 Swiss imports/exports
#### 1.2.1 Data Description
The FAO Database offers a variety of datasets containing information about the imports/exports quantities and values of every country for all years between 1961 and 2017.
##### Crops and livestock products
<details>
  <summary>Description</summary> 
  The trade database includes the following variables: export quantity, export value, import quantity, and import value. The trade database includes all food and agricultural products imported/exported annually by all the countries in the world.
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/Trade_Crops_Livestock_E_All_Data_(Normalized).zip
</details>

##### Live animals
<details>
  <summary>Description</summary> 
  The trade database includes the following variables: export quantity, export value, import quantity, and import value. The trade database includes all live animals imported/exported annually by all the countries in the world.
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/Trade_LiveAnimals_E_All_Data_(Normalized).zip
</details>

##### Detailed trade matrix
<details>
  <summary>Description</summary> 
   The trade database includes the following variables: export quantity, export value, import quantity and import value. The trade database includes all food and agricultural products as well as live animals imported/exported annually by all the countries in the world. It also includes the provenance/destination of each import/export.
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip
</details>

##### Trade Indices
<details>
  <summary>Description</summary> 
  Import/Exports Indices for every country. This only includes the year 2017 and fits the items into two categories.
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/Trade_Indices_E_All_Data_(Normalized).zip
</details>

Since we are interested in the provenance of each imports, we will mainly use the Detailed Trade Matrix. The Crops and Livestock products could be useful to us if we do not need the provenance. The Trade Indices dataset does not provide any useful information.

#### 1.2.2 Data Cleaning

In [13]:
"""Load Data"""
tradematrix_df = pd.read_csv('Data/FAOSTAT/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip', sep=',', encoding = "ISO-8859-1")

In [14]:
tradematrix_df.head()

,Reporter Country Code,Reporter Countries,Partner Country Code,Partner Countries,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5910,Export Quantity,2016,2016,tonnes,3.0,*
1,2,Afghanistan,4,Algeria,230,"Cashew nuts, shelled",5922,Export Value,2016,2016,1000 US$,23.0,*
2,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2015,2015,1000 US$,1.0,*
3,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2016,2016,1000 US$,1.0,*
4,2,Afghanistan,4,Algeria,1293,Crude materials,5922,Export Value,2017,2017,1000 US$,5.0,R


In [15]:
"""Extract Switzerland Trades"""
CH_code = 211 # Switzerland Area Code is 211
tradematrix_df_CH = tradematrix_df[tradematrix_df['Reporter Country Code'] == CH_code]

In [16]:
"""Check Units"""
tradematrix_df_CH['Unit'].value_counts()

1000 US$     297067
tonnes       291194
Head           3106
1000 Head       366
No               13
Name: Unit, dtype: int64

Again, we notice a few different units, the most frequent one being `tonnes` for quantity and `1000 US$` for value. We need to check what the other units correspond to to see if we should remove them.

In [17]:
"""Items expressed in Head, 1000 Head or No"""
tradematrix_df_CH[(tradematrix_df_CH['Unit'] == 'Head') | (tradematrix_df_CH['Unit'] == '1000 Head') | (tradematrix_df_CH['Unit'] == 'No')]['Item'].value_counts()

Horses               1586
Cattle                474
Chickens              271
Sheep                 239
Goats                 203
Asses                 188
Pigs                  184
Animals live nes      139
Camelids, other        45
Ducks                  36
Mules                  33
Rabbits and hares      31
Turkeys                28
Buffaloes              15
Beehives               13
Name: Item, dtype: int64

We see these units correspond to all live animals. As mentioned before, since we are interested in consumable food we do not need these items and thus can remove them. However, if we remove them if must make sure to remove also the live animals expressed in 1000 US$.

In [18]:
"""Create list of item codes corresponding to live animals"""
liveanimals_codes = tradematrix_df_CH[(tradematrix_df_CH['Unit'] == 'Head') | (tradematrix_df_CH['Unit'] == '1000 Head') | (tradematrix_df_CH['Unit'] == 'No')]['Item Code'].unique()

"""Remove rows with those items"""
tradematrix_df_CH = tradematrix_df_CH[tradematrix_df_CH['Item Code'].apply(lambda code: code in liveanimals_codes) == False]

In [19]:
"""Check units again"""
tradematrix_df_CH['Unit'].value_counts()

1000 US$    293575
tonnes      290668
Name: Unit, dtype: int64

Now that we have a more homogeneous dataset, we want to separate the four different elements i.e. Export/Import in terms of Quantity/Value.

In [20]:
"""Check different elements"""
tradematrix_df_CH['Element'].value_counts()

Import Value       177026
Import Quantity    175231
Export Value       116549
Export Quantity    115437
Name: Element, dtype: int64

In [21]:
"""Separate in four dataframes"""
imports_quant_df_CH = tradematrix_df_CH[tradematrix_df_CH['Element'] == 'Import Quantity']
imports_val_df_CH = tradematrix_df_CH[tradematrix_df_CH['Element'] == 'Import Value']
exports_quant_df_CH = tradematrix_df_CH[tradematrix_df_CH['Element'] == 'Export Quantity']
exports_val_df_CH = tradematrix_df_CH[tradematrix_df_CH['Element'] == 'Export Value']

Those four dataframes provide the swiss import/exports in terms of quantity and value from 1961 to 2017. It has no NaN values, only a lot of 0 values. 

#### 1.2.3 Pending Questions
* Flags: Are they useful? How would we use them?
* Items: Duplicates?
* Join the four dataframes to have the four values in each row? Then ignore flags?

---

### 1.4 Item Classification

In [22]:
"""Load Data"""
items_codes =pd.read_csv('Data/items_codes.csv', sep=',', encoding = "ISO-8859-1")
items_subgroups =pd.read_csv('Data/items_subgroups.csv', sep=',', encoding = "ISO-8859-1")
items_groups =pd.read_csv('Data/items_groups.csv', sep=',', encoding = "ISO-8859-1")

In [23]:
"""Filter and rename columns to obtain only what we need for classification"""
items_codes = items_codes[['Item Code', 'Item', 'Description']]
items_groups = items_groups[['Item Group Code', 'Item Group', 'Item Code']]
items_groups = items_groups.rename(columns={'Item Code': 'Item Subgroup Code'})
items_subgroups = items_subgroups[[ 'Item Code', 'Item', 'Description']]
items_subgroups = items_subgroups.rename(columns={'Item Code': 'Item Subgroup Code', 'Item': 'Item Subgroup', 'Description': 'Item Code'})

In [24]:
"""Obtain item codes for each subgroup"""
items_subgroups['Item Code'] = items_subgroups['Item Code'].apply(lambda desc : re.findall(r"\d+", str(desc)))

In [25]:
"""Remove subgroups that correspond to groups"""
items_subgroups = items_subgroups[items_subgroups['Item Subgroup'].apply(lambda item_subgroup: item_subgroup in items_groups['Item Group'].unique()) == False]

In [26]:
"""Explode item codes into separate rows"""
items_subgroups = items_subgroups.explode('Item Code')
items_subgroups = items_subgroups[items_subgroups['Item Code'].isna() == False]
items_subgroups['Item Code'] = items_subgroups['Item Code'].apply(lambda code: int(code))

In [27]:
"""Join the obtained dataframe to get full classification"""
items = items_codes.merge(items_subgroups, on='Item Code', how='left')
items = items.merge(items_groups, on='Item Subgroup Code', how='left')

In [28]:
items.head()

,Item Code,Item,Description,Item Subgroup Code,Item Subgroup,Item Group Code,Item Group
0,862,Alfalfa meal and pellets,"Alfalfa (lucerne) dried, whole, cut, chopped, ...",NaN,NaN,NaN,NaN
1,231,Almonds shelled,Around 55% of the weight in shell.,2551.0,Nuts and products,2901.0,Grand Total
2,231,Almonds shelled,Around 55% of the weight in shell.,2551.0,Nuts and products,2912.0,Treenuts
3,231,Almonds shelled,Around 55% of the weight in shell.,2551.0,Nuts and products,2903.0,Vegetal Products
4,221,"Almonds, with shell",Prunus amygdalus; P. communis; Amygdalus commu...,2551.0,Nuts and products,2901.0,Grand Total


---

### 1.3 Swiss Food Balance
#### 1.3.1 Data Description

##### Food Balance Sheets
<details>
  <summary>Description</summary> 
  
</details>

<details>
  <summary>Path</summary>
  
  Data/FAOSTAT/
</details>

In [32]:
"""Load Data"""
food_balance_df = pd.read_csv('Data/FAOSTAT/FoodBalanceSheets_E_All_Data_(Normalized).zip', encoding="ISO-8859-1")

In [33]:
"""Filter Data to obtain only what we need"""
food_balance_df_CH = food_balance_df[food_balance_df['Area Code'] == CH_code]

In [34]:
"""Spread the elements in 'Elements' to columns to be able to work on data"""
values_wide = food_balance_df_CH.pivot_table(index=['Area', 'Item', 'Year'], columns='Element', values='Value')
food_balance_df_CH_wide = food_balance_df_CH[['Area', 'Item', 'Year']]\
                    .merge(values_wide, right_index=True, left_on=['Area', 'Item', 'Year'])\
                    .drop_duplicates()
        
food_balance_df_CH_wide.head()

,Area,Item,Year,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,Food,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation,Total Population - Both sexes
7741481,Switzerland,Population,1961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5393.0
7741482,Switzerland,Population,1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5503.0
7741483,Switzerland,Population,1963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5618.0
7741484,Switzerland,Population,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5729.0
7741485,Switzerland,Population,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5830.0


In [48]:
"""Inspect the food supply and others per year"""
food_balance_df_CH_wide.groupby('Year').sum().head()

,Domestic supply quantity,Export Quantity,Fat supply quantity (g/capita/day),Feed,Food,Food supply (kcal/capita/day),Food supply quantity (kg/capita/yr),Import Quantity,Losses,Other uses,Processing,Production,Protein supply quantity (g/capita/day),Seed,Stock Variation,Total Population - Both sexes
Year,,,,,,,,,,,,,,,,
1961,15754.0,756.0,505.34,3799.0,9378.0,13815.0,1739.37,5185.0,451.0,208.0,1627.0,11140.0,351.95,291.0,185.0,5393.0
1962,16584.0,661.0,524.16,4290.0,9756.0,13459.0,1772.89,5565.0,429.0,240.0,1590.0,11569.0,332.71,285.0,117.0,5503.0
1963,16670.0,746.0,518.34,4433.0,9668.0,13161.0,1720.57,5438.0,415.0,267.0,1592.0,11485.0,338.38,297.0,495.0,5618.0
1964,17086.0,712.0,531.93,4316.0,9900.0,13326.0,1727.53,5943.0,398.0,265.0,1941.0,11907.0,336.65,275.0,-52.0,5729.0
1965,16876.0,723.0,521.01,4114.0,10166.0,13319.0,1743.16,6558.0,351.0,276.0,1717.0,10534.0,340.35,261.0,510.0,5830.0


## Food Balance Comments

We clearly see that some numbers were totally unexpected while inspecting `food_balance`. For example, having +13'000 kcal/capita/day seem too much since we know we are eating around 2000-3000 kcal/capita/day.
We did some researches and found that some elements in `Item` are the group of other elements, and hence we have some duplicates values.

To resolve this problem, we need to separate the group items from the items. As we already investigate on the items and their groups, we can use our results to split `food_balance` into 2 dataframes, one which contain the elements and another one which contain only the groups.

In [45]:
items_groups['Item Group'].unique()

array(['Alcoholic Beverages', 'Animal fats', 'Animal Products',
       'Aquatic Products, Other', 'Cereals - Excluding Beer', 'Eggs',
       'Fish, Seafood', 'Fruits - Excluding Wine', 'Grand Total', 'Meat',
       'Milk - Excluding Butter', 'Miscellaneous', 'Offals', 'Oilcrops',
       'Pulses', 'Spices', 'Starchy Roots', 'Stimulants',
       'Sugar & Sweeteners', 'Sugar Crops', 'Treenuts', 'Vegetable Oils',
       'Vegetables', 'Vegetal Products'], dtype=object)

In [51]:
items_subgroups['Item Subgroup'].unique()

array(['Alcohol, Non-Food', 'Apples and products',
       'Aquatic Animals, Others', 'Aquatic Plants', 'Bananas',
       'Barley and products', 'Beans', 'Beer', 'Beverages, Alcoholic',
       'Beverages, Fermented', 'Bovine Meat', 'Butter, Ghee',
       'Cassava and products', 'Cephalopods', 'Cereals, Other',
       'Citrus, Other', 'Cloves', 'Cocoa Beans and products',
       'Coconut Oil', 'Coconuts - Incl Copra', 'Coffee and products',
       'Cottonseed', 'Cottonseed Oil', 'Cream', 'Crustaceans', 'Dates',
       'Demersal Fish', 'Fats, Animals, Raw', 'Fish, Body Oil',
       'Fish, Liver Oil', 'Freshwater Fish', 'Fruits, Other',
       'Grapefruit and products', 'Grapes and products (excl wine)',
       'Groundnut Oil', 'Groundnuts (Shelled Eq)', 'Honey', 'Infant food',
       'Lemons, Limes and products', 'Maize and products',
       'Maize Germ Oil', 'Marine Fish, Other', 'Meat, Aquatic Mammals',
       'Meat, Other', 'Millet and products', 'Molluscs, Other',
       'Mutton & Goa

---

In [54]:
food_balance_df_CH_wide.Item.unique()

array(['Population', 'Grand Total', 'Vegetal Products', 'Animal Products',
       'Cereals - Excluding Beer', 'Wheat and products',
       'Rice (Milled Equivalent)', 'Barley and products',
       'Maize and products', 'Rye and products', 'Oats',
       'Millet and products', 'Sorghum and products', 'Cereals, Other',
       'Starchy Roots', 'Cassava and products', 'Potatoes and products',
       'Sweet potatoes', 'Yams', 'Roots, Other', 'Sugar Crops',
       'Sugar cane', 'Sugar beet', 'Sugar & Sweeteners',
       'Sugar (Raw Equivalent)', 'Sweeteners, Other', 'Honey', 'Pulses',
       'Beans', 'Peas', 'Pulses, Other and products', 'Treenuts',
       'Nuts and products', 'Oilcrops', 'Soyabeans',
       'Groundnuts (Shelled Eq)', 'Sunflower seed',
       'Rape and Mustardseed', 'Cottonseed', 'Coconuts - Incl Copra',
       'Sesame seed', 'Palm kernels', 'Olives (including preserved)',
       'Oilcrops, Other', 'Vegetable Oils', 'Soyabean Oil',
       'Groundnut Oil', 'Sunflowerseed Oil'